# Análise de e clusterização de tempo por quantidade de PC

&nbsp;&nbsp;&nbsp;&nbsp;Esse é um arquivo python notebook (.ipynb) para uma compreensão de dados para a parceria com a Compass. São utilizadas as bibliotecas pandas, numpy e matplotlib. É feito o carregamento e concatenação simultânea de cada tabela (.csv) contendo de dados dos meses. É utilizada uma função para descrever a estatística descritiva da coluna "datetime" da tabela. Em seguida foram criados gráficos para diferentes análises da coluna.

In [12]:
import pandas as pnd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline


&nbsp;&nbsp;&nbsp;&nbsp;Carregamento e concatenação de todas tabelas disponíveis

In [ ]:
df = pnd.read_csv('month_2.csv')# Concatena diferentes tabelas com cada mês (dados)
# Em seguida salva todos os dados em um único DataFrame chamado df
listas = ['month_2.csv', 'month_3.csv', 'month_4.csv', 'month_5.csv', 'month_6.csv']
df = []
for arquivo in listas:
    df += [pnd.read_csv(arquivo)]

# Concatena todos os dataframes em um único dataframe chamado df
df = pnd.concat(df)

# Chama o dataframe contido na variável chamada df
df.sort_values(by='datetime', inplace=True)
df

### Pré processamento de dados

&nbsp;&nbsp;&nbsp;&nbsp;Compreensão de dados da coluna datetime para verificar se a análise descritiva possui alguma incoerência que precisa ser corrigida

In [14]:
# Essa linha de código converte a coluna 'datetime' do DataFrame df para o tipo datetime usando a função to_datetime da biblioteca pandas (pnd).
df['datetime'] = pnd.to_datetime(df['datetime'])

In [ ]:
# Visualiza as análises descritivas da coluna datetime
df['datetime'].describe()

&nbsp;&nbsp;&nbsp;&nbsp;Lê os dados cadastrais e forma um novo dataframe

In [ ]:
# Carrega e salva dadosCadastrais como dadosCadastrais
dadosCadastrais = pnd.read_csv('informacao_cadastral.csv')

# Chama o dataframe contido na variável dadoCadastrais
dadosCadastrais

Cria uma função extendida de descrever (estatísticas descritivas)

In [17]:
# Cria uma função extendida de descrever (estatísticas descritivas)
def describeExtended(data):
    description = data.describe()

    # Adiciona a mediana, variância e moda para a função describe	
    description.loc['var'] = data.var()
    description.loc['median'] = data.median()
    description.loc['mode'] = data.mode().iloc[0]
    return description

In [ ]:
# Estatística descritiva pulseCount
print(describeExtended(df.pulseCount))

&nbsp;&nbsp;&nbsp;&nbsp;Nessa seção são feitas algumas alterações básicas e geradas uma nova dataframe a partir do que foi filtrado, organizado (ordenar por data), e calculadas as diferenças entre valores.

In [ ]:
# Filtra os usuários que estão consumindo gás (operacionais) e por seu código de cliente
usuariosUnicos = dadosCadastrais[dadosCadastrais.situacao == 'CONSUMINDO GÁS']['clientCode'].unique() 
# Organiza os dados dos usuários filtrados pela data
mesFiltrado = df[df['clientCode'].isin(usuariosUnicos)].sort_values(by='datetime') 
# Filtra meterSN diferente de '>N<A'
mesFiltrado = mesFiltrado[mesFiltrado['meterSN'] != '>N<A']
# Cria uma nova variável mesFiltrado, agrupa por meterSN e clientCode e seleciona a primeira linha
resultado = mesFiltrado.groupby(['meterSN', 'clientCode']).first() 
# Seleciona a coluna pulseCount
resultado = resultado[['pulseCount']] 
# Cria um novo dataframe com a coluna pulseCountInicial
final = pnd.DataFrame({'pulseCountInicial': resultado.pulseCount}) 
# Junta os dataframes
merged_df = pnd.merge(df, final, on=['meterSN', 'clientCode'], how='left') 
# Calcula a diferença entre pulseCount e pulseCountInicial
merged_df['pulseCount'] = merged_df['pulseCount'] - merged_df['pulseCountInicial'] 
#Mostra o dataframe 'merged_df'
merged_df.sort_values(by='datetime', inplace=True)
merged_df

### Clusterização do horário de cada Pulse Count

In [ ]:
# Garante que a coluna 'datetime' esteja no formato datetime
merged_df['datetime'] = pnd.to_datetime(merged_df['datetime'])

# Extrai a hora da coluna `datetime`
merged_df['hour'] = merged_df['datetime'].dt.hour

# Cria a coluna `period` com base na hora
def get_period(hour):
  if 0 <= hour <= 6:
    return 'madrugada'
  elif 7 <= hour <= 12:
    return 'manhã'
  elif 13 <= hour <= 18:
    return 'tarde'
  else:
    return 'noite'

merged_df['period'] = merged_df['hour'].apply(get_period)

# Imprime os primeiros 5 registros para verificar
print(merged_df.head().to_string())

In [ ]:
# Forma uma lista com todos os números de Pulse Count por hora
merged_df['hour'].value_counts().sort_index()

### Gráficos

&nbsp;&nbsp;&nbsp;&nbsp;Ajuda a visualizar como está a distribuição de registro de PC ao longo de um dia inteiro pegando informações de todas as tabelas

&nbsp;&nbsp;&nbsp;&nbsp;Gráfico de barras que apresenta a quantidade de total, ou seja, pega todas as medições de pulse count que aconteceram em cada hora do dia e soma juntando tudo.

In [ ]:
# Pega a lista acima e transforma em um gráfico de barras
ax = merged_df['hour'].value_counts().sort_index().plot(kind='bar')

&nbsp;&nbsp;&nbsp;&nbsp;Pega os horários do gráfico anterior e clusteriza todos em 4 seções, madrugada, manhã, tarde e noite

In [ ]:

merged_df['period'].value_counts().sort_index().plot(kind='bar')

&nbsp;&nbsp;&nbsp;&nbsp;Com base nos gráficos acima, podemos chegar a uma conclusão central: embora os usuários utilizem gás em horários "comuns", como 06:00 (horário em que os brasileiros costumam acordar para ir trabalhar), 12:00 (horário de almoço) e 20:00 (horário comum para o jantar ou para tomar banho), as medições ocorrem, de fato, após um período de tempo, o que resulta em uma distribuição mais uniforme ao longo das horas. No entanto, ao analisarmos o período da manhã, podemos observar uma distribuição "anormal" ao longo do horário, que difere dos demais, principalmente às 09:00, quando o pico de registro de consumo é atingido. Isso pode nos levar a duas questões: os registros de PC são realizados com atraso, mas sem um horário exato de delay; e a temperatura mais baixa pela manhã poderia levar a um maior uso de aquecedores a gás, resultando nesse pico de consumo.